In [ ]:
import os
from PIL import Image
import numpy as np
from CLIP import clip
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import json

In [ ]:
from torchvision.datasets import MNIST
mnist = MNIST(root=os.path.expanduser("~/.cache"), download=True, train=True)
class YourDataset(Dataset):         #每一个类抽10个，如何实现？
    def __init__(self,dataset,preprocess,sam_num):

        self.img_process = preprocess

        self.samples = []
        self.sam_labels = []
        self.samid=[]

        catcount=[0,0,0,0,0,0,0,0,0,0]
        for image,cl_id in dataset:

            if(catcount[cl_id]<sam_num):
                label=dataset.classes[cl_id]
                label = "a photo of " + label
                self.samples.append(image)
                self.sam_labels.append(label)
                self.samid.append(cl_id)
                catcount[cl_id]+=1
            if(all(item >sam_num-1 for item in catcount)):
                break
        self.tokens = clip.tokenize(self.sam_labels)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image = self.samples[idx]
        token = self.tokens[idx]

        image = self.img_process(image)
        return image,token,self.samid[idx]

device = torch.device("cuda:0")
net, preprocess = clip.load("ViT-B/32",device=device,jit=False)
sam_num=5
datasets=YourDataset(mnist,preprocess,sam_num)
net.eval()

In [ ]:
def aux_cstr(datasets,num_cls):
    i=0
    

    for image,_,label in datasets:
        image = image.unsqueeze(0).to(device)
        image=net.encode_image(image)

        if i==0:
            temp=image.to(device)
            templ=torch.zeros([1,num_cls]).to(device)
            templ[0,label]+=1
            i+=1
        else:
            temp=torch.cat([temp,image.to(device)])
            templl=torch.zeros([1,num_cls]).to(device)
            templl[0,label]+=1
            templ=torch.cat([templ,templl])
    return temp.T,templ
        
vresult,lresult=aux_cstr(datasets,10)
print(vresult.shape)
print(lresult.shape)

    

In [ ]:
class TestDataset(Dataset):         #每一个类抽10个，如何实现？
    def __init__(self,dataset,preprocess):

        self.img_process = preprocess

        self.samples = []
        self.sam_labels = []
        self.samid=[]

        for image,cl_id in dataset:
            label=dataset.classes[cl_id]
            label = "a photo of " + label
            self.samples.append(image)
            self.sam_labels.append(label)
            self.samid.append(cl_id)
        self.tokens = clip.tokenize(self.sam_labels)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image = self.samples[idx]
        token = self.tokens[idx]
        image = self.img_process(image)
        return image,token,self.samid[idx]
test_set= MNIST(root=os.path.expanduser("~/.cache"), download=True, train=False)
test_set=TestDataset(test_set,preprocess)
test_set=DataLoader(dataset=test_set,batch_size=1,shuffle=False,num_workers=4,pin_memory=False)


In [ ]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in mnist.classes]).to(device)
# text_features = net.encode_text(text_inputs)
# print(text_features.shape)

In [ ]:

def zero_shot_wrapper(test_set,vresult,lresult,text_inputs):
    count=0
    right=0
    with torch.no_grad():
        text_features = net.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    with torch.cuda.amp.autocast(enabled=True):
        for images,labels,cl_id in test_set:
            for i in range(5):
                torch.cuda.empty_cache()
            count+=1
            images = images.to(device)
            with torch.no_grad():
                images=net.encode_image(images)

                images /= images.norm(dim=-1, keepdim=True)
                vresult /= vresult.norm(dim=-1, keepdim=True)

                logits=1.0*torch.exp(-5.5*(1-images@vresult))@lresult+images@(text_features.T)
            values, indices = logits.topk(1)
            if int(indices)==cl_id:
                right+=1
                print("count:",count,"right:",right)


        
zero_shot_wrapper(test_set,vresult,lresult,text_inputs)